<a href="https://colab.research.google.com/github/Juuliee/ABSA-PyTorch/blob/master/entity_aspect_pair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Import Packages

In [2]:
import pandas as pd
import numpy as np

## Import Data

without unnecessary rows and columns.

In [123]:
original_data = pd.read_excel('/content/drive/My Drive/Review Classifier - Trainingsset-CCG-3.xlsx')

In [124]:
print(original_data[original_data.duplicated(keep= False)]) # row 194 and 202 duplicate

       Review Aspect Aspect 1  ... Aspect 3 Sentiment Aspect 3 Sub-category Aspect 3
194  Gute app    NaN      NaN  ...      NaN                NaN                   NaN
202  Gute app    NaN      NaN  ...      NaN                NaN                   NaN

[2 rows x 11 columns]


In [125]:
def drop_rows(my_data):

    lorem_indices = my_data[my_data["Feedback"] == ' Lorem ipsum dolor sit amet\\\\  consectetur adipiscing elit. '].index
    new_data = my_data.drop(lorem_indices)
    new_data.drop_duplicates(inplace=True,ignore_index=True)
    new_data.reset_index(drop=True, inplace=True)

    return new_data

In [126]:
entity_data = original_data.rename(columns={"Review": "Feedback", "Aspect 1": "Aspect_1", "Sentiment Aspect 1": "Sentiment_1", "Aspect 2": "Aspect_2", "Sentiment Aspect 2": "Sentiment_2", "Aspect 3": "Aspect_3", "Sentiment Aspect 3": "Sentiment_3", "Sub-category Aspect 1": "Entity_1", "Sub-category Aspect 2": "Entity_2", "Sub-category Aspect 3": "Entity_3"})
entity_data = drop_rows(entity_data)

## Remove special characters

In [127]:
def remove_special_characters(my_data):

    my_data["Feedback"] = my_data["Feedback"].str.replace('_',' ').str.replace('\\\\','')
    my_data["Feedback"] = my_data["Feedback"].str.replace('<br>',' ')
    my_data["Feedback"] = my_data["Feedback"].str.lstrip().str.rstrip() # remove leading and trailing whitespaces

    return my_data

In [128]:
entity_data = remove_special_characters(entity_data)

## Filter data

Use only entity-relevant rows.


In [184]:
entity_train =entity_data[~entity_data.Entity_1.isnull()]
entity_train.reset_index(drop=True, inplace=True)

,Feedback,Aspect,Aspect_1,Sentiment_1,Entity_1,Aspect_2,Sentiment_2,Entity_2,Aspect_3,Sentiment_3,Entity_3
0,Ich war bei einem großen deutschen Versicherun...,"Beiträge,Geld",Beitrag,neutral,Preis,Geld,positive,Preis,NaN,NaN,NaN
1,Lt. HUK-Schreiben benötige ich die Super-PIN u...,"Bereich,Problem",Bereich,neutral,Beratung,Problem,negative,Problemlösung,NaN,NaN,NaN
2,Bisher eine unkomplizierte Abwicklung - auch i...,"Abwicklung,Schadensfall",Abwicklung,positive,Leistung,Schadensfall,positive,Leistung,NaN,NaN,NaN
3,Schnelle Beratung und Erstellung der eVB samt...,"Beratung,Unterlagen",Beratung,positive,Beratung,Unterlagen,positive,Erreichbarkeit,NaN,NaN,NaN
4,Der Service der RechnungsApp ist super. Die Be...,"Service,Bearbeitung",Service,positive,Erreichbarkeit,Bearbeitung,positive,Erreichbarkeit,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
578,Die telefonische Erreichbarkeit könnte verbess...,Erreichbarkeit,Erreichbarkeit,negative,Erreichbarkeit,Freundlichkeit,negative,Freundlichkeit,NaN,NaN,NaN
579,Die Preissteigerung nach einem Unfall erstaunt...,"Unfall,Versicherung,Formular",Unfall,negative,Preis,Versicherung,negative,Problemlösung,Formular,negative,Problemlösung
580,Sehr hohe Summe die ich nach zahlen musste...,NaN,Summe,negative,Preis,NaN,NaN,NaN,NaN,NaN,NaN
581,Die grüne Versichertenkarte wurde fehlerhaft a...,NaN,Karte,negative,Qualität,Preis,positive,Preis,NaN,NaN,NaN


In [ ]:
len(entity_train)

In [189]:
ii = 0
(my_data["Entity_2"][ii]+','+my_data["Entity_1"][ii])

'Preis,Preis'

In [239]:
for ii in range(len(my_data)):
    if my_data["Entity_3"][ii].isnull() == False:
        my_data["Entity_Tuple"][ii] = (my_data["Entity_3"][ii]+','+my_data["Entity_2"][ii]+','+my_data["Entity_1"][ii])
    if type(my_data["Entity_2"][ii]) == str:
        my_data["Entity_Tuple"][ii] = (my_data["Entity_2"][ii]+','+my_data["Entity_1"][ii])
    if type(my_data["Entity_1"][ii]) == str:
        my_data["Entity_Tuple"][ii] = (my_data["Entity_1"][ii])
    else:
        my_data = my_data.drop(ii)

my_data.reset_index(inplace=True, drop=True)

0

In [240]:
my_data["Entity_3"][ii].isnull()
my_data[34:36]
type(my_data["Entity_3"][35])

AttributeError: ignored

In [245]:
def add_entity_tuple(my_data):
    
    none_list = [None] * len(my_data)
    my_data["Entity_Tuple"] = none_list

    for ii in range(len(my_data)):
        if type(my_data["Entity_3"][ii]) == str:
           if type(my_data["Entity_2"][ii]) == str:
              my_data["Entity_Tuple"][ii] = (my_data["Entity_3"][ii]+','+my_data["Entity_2"][ii]+','+my_data["Entity_1"][ii])
        if type(my_data["Entity_2"][ii]) == str:
            my_data["Entity_Tuple"][ii] = (my_data["Entity_2"][ii]+','+my_data["Entity_1"][ii])
        elif type(my_data["Entity_1"][ii]) == str:
            my_data["Entity_Tuple"][ii] = (my_data["Entity_1"][ii])
        else:
            my_data = my_data.drop(ii)

    my_data.reset_index(inplace=True, drop=True)

    return my_data

In [ ]:
entity_train = add_entity_tuple(entity_train)

# Count aspect, entities and entity-aspect pairs

In [132]:
def aspect_tuple_generator(my_data):

    sep_aspects = []
    tuple_list = []

    for ii in range(len(my_data)):

        if type(my_data.Aspect[ii]) == str:
            sep_aspects += my_data["Aspect"][ii].lower().split(sep=',')
            # split list of aspects and make a tuple out of them
            tuple_list += [tuple(my_data.Aspect[ii].lower().split(sep=','))]
        else:
            # add aspect "no_aspect" to reviews without any aspect
            tuple_list += [tuple(["no_aspect", ])]
          
    return sep_aspects, tuple_list

In [ ]:
sep_aspects, aspects_in_tuples = aspect_tuple_generator(entity_data)

In [170]:
len(set(sep_aspects))

196

###Get List of rare Aspects

In [138]:
def aspect_list_cleaner(min_appearance_freq):

    # create a frequency list for aspects and separate into two lists: 
    # cleaned_aspect_list & rare_aspect_list

    aspect_freq_list = [[x,sep_aspects.count(x)] for x in set(sep_aspects)]

    cleaned_aspect_list = [] 
    rare_aspect_list = []

    for ii in range(len(aspect_freq_list)):

        # each aspect has to appear at least min_appearance_freq time within
        # the data to stay in the cleaned_aspect_list
        # if min_appearance_freq = 1, no aspects are removed
        if aspect_freq_list[ii][1] < min_appearance_freq: 
            rare_aspect_list += aspect_freq_list[ii][0:-1]
        else:
            cleaned_aspect_list += aspect_freq_list[ii][0:-1]
    
    return aspect_freq_list, cleaned_aspect_list, rare_aspect_list

In [ ]:
#Choose how often an aspect has to appear at least.
MIN_APP_FREQ = 1
aspect_freq_list, cleaned_aspect_list, rare_aspect_list = aspect_list_cleaner(MIN_APP_FREQ)
# Check length of separated lists
[len(cleaned_aspect_list),len(rare_aspect_list)]

## Neuer Abschnitt

In [160]:
nones = np.zeros(shape=(0,10))
data = pd.DataFrame(nones, columns=["Aspect", "Aspect_Freq", "Beratung", "Erreichbarkeit", "Freundlichkeit", "Kompetenz", "Qualität", "Preis","Leistung/Nutzung", "Anforderung"])

,Aspect,Aspect_Freq,Beratung,Erreichbarkeit,Freundlichkeit,Kompetenz,Qualität,Preis,Leistung/Nutzung,Anforderung


In [180]:
data2 =pd.DataFrame(aspect_freq_list,columns=['Aspect','Aspect_Freq'])
data.Aspect =data2.assign(Beratung ="", Erreichbarkeit="", Freundlichkeit="", Kompetenz="", Qualität="", Preis="",Leistung_Nutzung="", Anforderung="")

,Aspect,Aspect_Freq,Beratung,Erreichbarkeit,Freundlichkeit,Kompetenz,Qualität,Preis,Leistung_Nutzung,Anforderung
0,aufbau,1,,,,,,,,
1,betrag,3,,,,,,,,
2,beschreibung,1,,,,,,,,
3,hilfe,6,,,,,,,,
4,bearbeitung,12,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
191,suche,1,,,,,,,,
192,anbieter,7,,,,,,,,
193,nutzen,1,,,,,,,,
194,nachfrage,3,,,,,,,,


In [183]:
for ii in range(len(entity_train)):
  if entity_train.

583

##Remove rare aspects from tuple List

In [142]:
def rare_aspect_remover(tuple_list, rare_aspect_list):

    # create a new tuple_list: cleaned_aspects, removing rare aspects that just appear once
    cleaned_aspects = [None] * len(tuple_list)

    for ii in range(len(tuple_list)):
        cleaned_aspects[ii] = tuple()
        for ind in tuple_list[ii]:
            if ind not in rare_aspect_list:
                cleaned_aspects[ii] += tuple([ind])
        # if the removed aspect was the only aspect, we need to label it "no_aspect"
        if cleaned_aspects[ii] == ():
            cleaned_aspects[ii] = tuple(["no_aspect", ])
    
    return cleaned_aspects

In [ ]:
cleaned_aspects = rare_aspect_remover(aspects_in_tuples, rare_aspect_list)

Add columns to data set

In [ ]:
def aspect_encoder(my_data, cleaned_aspects):

    none_list = [None] * len(cleaned_aspects)
    tuple_df = pd.DataFrame(none_list, columns=["Tuples"])
    tuple_df.Tuples = cleaned_aspects

    my_data["Cleaned_Aspects"] = tuple_df.Tuples

    mlb = MultiLabelBinarizer()
    encoded_aspects = mlb.fit_transform(my_data["Cleaned_Aspects"])
    aspect_table = pd.DataFrame(encoded_aspects, columns=mlb.classes_)

    my_data["Aspect_Encoding"] = aspect_table.values.tolist()

    return my_data, aspect_table, mlb.classes_

In [ ]:
kfz_data, aspect_table, aspects = aspect_encoder(kfz_data, cleaned_aspects)